<a href="https://colab.research.google.com/github/alyrazik/News_content_collect_store/blob/MongoDB-Saving-and-querying/CrawlingBBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  News Content Collect and Store
## by Aly Abdelrazek
 
 


##Setup and Importing libraries

In [1]:
#!pip install dnspython
!python -m pip install pymongo[srv]
#!python -m pip install pymongo[snappy,gssapi,srv,tls]

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import pymongo
from pymongo import MongoClient
from datetime import datetime


In [3]:
pymongo.version

'3.11.0'

##Global variables

In [4]:
news_page_url = "https://bbc.com/news"
absolute_link_prefix = "https://bbc.com"


##Crawl


In [5]:
# Obtains news stories links from news_page_url (https://bbc.com/news)
def news_page_to_news_urls(url, prefix):
    ''' Takes the URL to a news page and the prefix of the absolute link, and returns a list of news stories URLs.'''
    news_urls = []
    news_page_html = requests.get(url).text
    news_page_soup = BeautifulSoup(news_page_html, "html.parser")
    news_page_links = news_page_soup.findAll("a") #find all links
    
    for link in news_page_links:
        href = link.get("href")
        if href.startswith("/news") and href[-1].isdigit(): #news articles start with /news and ends with a digit
            news_url = prefix + href
            s = BeautifulSoup(requests.get(news_url).text , "html.parser")
            if s.find(class_= "css-16rg7hm-ContainerWithSidebarWrapper e1jl38b40") is not None: #this class is only available in news articles
              news_urls.append(news_url)
    return news_urls

news_urls = news_page_to_news_urls(news_page_url, absolute_link_prefix)

In [6]:
(news_urls)

['https://bbc.com/news/world-europe-54729957',
 'https://bbc.com/news/world-europe-54729957',
 'https://bbc.com/news/world-europe-54602171',
 'https://bbc.com/news/world-asia-54711452',
 'https://bbc.com/news/world-asia-54728738',
 'https://bbc.com/news/world-asia-54701279',
 'https://bbc.com/news/world-asia-china-54729054',
 'https://bbc.com/news/business-54725681',
 'https://bbc.com/news/business-54728298',
 'https://bbc.com/news/world-australia-54728348',
 'https://bbc.com/news/world-asia-54716077',
 'https://bbc.com/news/world-europe-54728893',
 'https://bbc.com/news/world-asia-54716077',
 'https://bbc.com/news/world-europe-54728893',
 'https://bbc.com/news/election-us-2020-54727919',
 'https://bbc.com/news/science-environment-54714692',
 'https://bbc.com/news/election-us-2020-54504585',
 'https://bbc.com/news/election-us-2020-53657174',
 'https://bbc.com/news/world-australia-54716546',
 'https://bbc.com/news/science-environment-54721921',
 'https://bbc.com/news/election-us-2020-54

## Cleanse

In [61]:
#news_indices = [str(i) for i in range(len(news_urls))]
news_headings = []
news_texts =[]
news_times = []

for url in news_urls:
    print("Fetching {}".format(url))
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    news_heading = (soup.find('h1')).text
    news_text = [p.text for p in soup.find('article').find_all('p')]
    news_time = (soup.find('time'))['datetime']

    news_headings.append(news_heading)
    news_times.append(np.datetime64(news_time))
    news_texts.append(news_text)

Fetching https://bbc.com/news/world-europe-54729957


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


Fetching https://bbc.com/news/world-europe-54729957
Fetching https://bbc.com/news/world-europe-54602171
Fetching https://bbc.com/news/world-asia-54711452
Fetching https://bbc.com/news/world-asia-54728738
Fetching https://bbc.com/news/world-asia-54701279
Fetching https://bbc.com/news/world-asia-china-54729054
Fetching https://bbc.com/news/business-54725681
Fetching https://bbc.com/news/business-54728298
Fetching https://bbc.com/news/world-australia-54728348
Fetching https://bbc.com/news/world-asia-54716077
Fetching https://bbc.com/news/world-europe-54728893
Fetching https://bbc.com/news/world-asia-54716077
Fetching https://bbc.com/news/world-europe-54728893
Fetching https://bbc.com/news/election-us-2020-54727919
Fetching https://bbc.com/news/science-environment-54714692
Fetching https://bbc.com/news/election-us-2020-54504585
Fetching https://bbc.com/news/election-us-2020-53657174
Fetching https://bbc.com/news/world-australia-54716546
Fetching https://bbc.com/news/science-environment-547

In [91]:
df = pd.DataFrame(list(zip( news_urls, news_times, news_headings, news_texts)), 
                  columns =['URL', 'Date_Time', 'Heading', 'Text']) 

In [92]:
df

,URL,Date_Time,Heading,Text
0,https://bbc.com/news/world-europe-54729957,2020-10-29 12:17:08,France attack: Deadly stabbing in Nice 'terror...,[.css-14iz86j-BoldText{font-weight:bold;}Three...
1,https://bbc.com/news/world-europe-54729957,2020-10-29 12:17:08,France attack: Deadly stabbing in Nice 'terror...,[.css-14iz86j-BoldText{font-weight:bold;}Three...
2,https://bbc.com/news/world-europe-54602171,2020-10-19 23:16:08,Samuel Paty: Beheading of teacher deepens divi...,"[By Lucy WilliamsonParis correspondent, .css-1..."
3,https://bbc.com/news/world-asia-54711452,2020-10-29 00:35:53,Al-Qaeda still 'heavily embedded' within Talib...,"[By Secunder KermaniBBC News, .css-14iz86j-Bol..."
4,https://bbc.com/news/world-asia-54728738,2020-10-29 04:42:11,Coronavirus: Remote Marshall Islands records f...,[.css-14iz86j-BoldText{font-weight:bold;}The M...
5,https://bbc.com/news/world-asia-54701279,2020-10-29 00:29:48,Muttiah Muralitharan on the 'challenge' of his...,[.css-14iz86j-BoldText{font-weight:bold;}Forme...
6,https://bbc.com/news/world-asia-china-54729054,2020-10-29 11:22:04,Tony Chung: Hong Kong activist detained near U...,[.css-14iz86j-BoldText{font-weight:bold;}Hong ...
7,https://bbc.com/news/business-54725681,2020-10-28 21:30:36,"US tries to block Ngozi Okonjo-Iweala, who wou...",[.css-14iz86j-BoldText{font-weight:bold;}The a...
8,https://bbc.com/news/business-54728298,2020-10-29 03:12:37,Samsung sees sales surge from rival Huawei's ban,[.css-14iz86j-BoldText{font-weight:bold;}Samsu...
9,https://bbc.com/news/world-australia-54728348,2020-10-29 04:12:22,Adriana Rivas: Pinochet-era kidnap suspect 'el...,[.css-14iz86j-BoldText{font-weight:bold;}A Chi...


##Saving to MongoDB

In [93]:
#connect to client
client = MongoClient("mongodb+srv://aly:a@cluster0.4pfcp.mongodb.net/db?retryWrites=true&w=majority")

In [94]:
#create a database
db = client["news_database"]

In [95]:
#create a collection (a table)
bbc_news = db["BBC_news"]

In [97]:
df.T.head().to_dict('list')

{0: ['https://bbc.com/news/world-europe-54729957',
  Timestamp('2020-10-29 12:17:08'),
  "France attack: Deadly stabbing in Nice 'terrorist attack', mayor says",
  ['.css-14iz86j-BoldText{font-weight:bold;}Three people have been killed in a knife attack at a church in the French city of Nice, police say.',
   'Nice Mayor Christian Estrosi said there had been a "terrorist attack at the heart of the Notre-Dame basilica".',
   'One elderly victim who had come to pray was "virtually beheaded". A suspect was shot and detained shortly afterwards.',
   'Mr Estrosi spoke of "Islamo-fascism" and that the suspect had "repeated endlessly \'Allahu Akbar\' (God is greatest)."',
   "President Emmanuel Macron is on his way to the scene. France's national anti-terror prosecutors have opened a murder inquiry.",
   'Mr Estrosi compared the attack to the recent murder of teacher Samuel Paty, who was beheaded close to his school outside Paris earlier this month.',
   "Police have not suggested a motive fo

In [100]:
#df.set_index('Heading', inplace=True)
#data_dict = df.T.to_dict('list')
bbc_news.insert_one(data_dict)

##Retrieving from MongoDB